In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [3]:
X = torch.rand(2, 20)

In [4]:
X

tensor([[0.2618, 0.6730, 0.6457, 0.1509, 0.3658, 0.4733, 0.4626, 0.6783, 0.0777,
         0.8618, 0.2652, 0.5702, 0.4045, 0.6780, 0.8939, 0.5390, 0.1306, 0.5186,
         0.3443, 0.4346],
        [0.2490, 0.3038, 0.7080, 0.5155, 0.8175, 0.0156, 0.2467, 0.2734, 0.6699,
         0.8647, 0.6870, 0.8963, 0.6157, 0.5763, 0.2196, 0.2894, 0.8135, 0.2787,
         0.2915, 0.3722]])

In [5]:
net(X)

tensor([[-0.0555,  0.1592,  0.0068, -0.0728, -0.0468, -0.0437,  0.1536, -0.1071,
         -0.1221,  0.0384],
        [-0.0398,  0.0918, -0.0031, -0.1778,  0.0468, -0.0741,  0.2317, -0.1145,
         -0.0448,  0.0180]], grad_fn=<AddmmBackward>)

In [12]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 2)
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

In [13]:
net = MLP()

In [14]:
net(X)

tensor([[-0.0877, -0.0414],
        [-0.0093,  0.0443]], grad_fn=<AddmmBackward>)

In [16]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block
    
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

In [24]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

In [25]:
net(X)

tensor([[ 0.1560,  0.0248,  0.0800,  0.0722,  0.1611,  0.1110,  0.1316,  0.0190,
         -0.0278,  0.1909],
        [ 0.1741,  0.1360, -0.0040,  0.0381, -0.0191,  0.3031,  0.1588, -0.0627,
          0.0177,  0.1225]], grad_fn=<AddmmBackward>)

In [27]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [30]:
net = FixedHiddenMLP()

In [31]:
net(X)

tensor(-0.1820, grad_fn=<SumBackward0>)

In [33]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(), nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)
    def forward(self, X):
        return self.linear(self.net(X))
    


In [37]:
chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2661, grad_fn=<SumBackward0>)

In [39]:
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand((2, 4))

In [40]:
net(X)

tensor([[-0.3586],
        [-0.1628]], grad_fn=<AddmmBackward>)

In [47]:
net[2].bias.data

tensor([-0.1612])

In [52]:
net[2].weight.grad == None

True

In [50]:
net[0].weight.grad == None

True

In [56]:
net.state_dict()['2.weight'].data

tensor([[-0.0232,  0.2654, -0.3441, -0.2445, -0.0200, -0.1198, -0.1132, -0.0345]])

In [57]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        net.add_module(f'block {i}', block1())
    return net

In [60]:
rgnet = nn.Sequential(block2(), nn.Linear(4, 1))

In [61]:
rgnet(X)

tensor([[-0.1442],
        [-0.1442]], grad_fn=<AddmmBackward>)

In [62]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [66]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01)
        nn.init.ones_(m.bias)
net.apply(init_normal)


Sequential(
  (0): Linear(in_features=4, out_features=8, bias=True)
  (1): ReLU()
  (2): Linear(in_features=8, out_features=1, bias=True)
)

In [67]:
net[0].weight.data[0], net[0].bias.data[0]

(tensor([-0.0062,  0.0008, -0.0198,  0.0025]), tensor(1.))

In [71]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.ones_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(1.))

In [72]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
        
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

In [73]:
net[0].apply(xavier)
net[2].apply(init_42)

Linear(in_features=8, out_features=1, bias=True)

In [74]:
net[0].weight.data[0]

tensor([-0.4542, -0.0723,  0.4966, -0.2233])

In [75]:
net[2].weight.data[0]

tensor([42., 42., 42., 42., 42., 42., 42., 42.])

In [76]:
# 参数绑定
shared = nn.Linear(8, 8)
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), shared, nn.ReLU(), shared, nn.ReLU(), nn.Linear(8 ,1))

In [77]:
net(X)

tensor([[-0.0505],
        [-0.0488]], grad_fn=<AddmmBackward>)

In [79]:
# 自定义层
class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()
        
    def forward(self, X):
        return X - X.mean()

In [82]:
layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5, 6, 7]))

tensor([-3., -2., -1.,  0.,  1.,  2.,  3.])